In [1]:
import cv2
import time
import os
import HandTrackingModule as htm    # 07. 導入自訂的HandTrackingModule.py

# 01. 取得鏡頭影像的基本架構
wCam, hCam = 640, 480

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

# 02. 設定手指圖片路徑
folderPath = "Fingerimages"
myList = os.listdir(folderPath)
print(myList)

# 03. 載入圖片至overlayList中
overlayList = []
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    print(f'{folderPath}/{imPath}')
    overlayList.append(image)
print(len(overlayList))

# 05. FPS時間設定
pTime = 0

# 07. 使用 HandTrackingModule.py函式擷取手部資訊
detector = htm.handDetector(detectionCon=0.75, maxHands=1)


while True:
    success, img = cap.read()
    
    # 04. 水平翻轉畫面
    img = cv2.flip(img, 1) 
    
    # 07. 偵測手並傳回座標位置
    img = detector.findHands(img, draw = False)
    lmList, bbox = detector.findPosition(img)
    
    # 10. 偵測手並傳回座標位置
    if len(lmList) != 0: 
        # 10. 檢查那些手指向上
        fingers = detector.fingersUp()
        #print(fingers)
        # 10. 計算向上手指數量
        totalFingers = fingers.count(1)   
        # 10. 輸出數字
        cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
        cv2.putText(img, f'{int(totalFingers)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)

        # 11. 修改原4.載入圖片至影像中的程式並放入if len(lmList) != 0:中
        h, w, c = overlayList[totalFingers-1].shape
        img[0:h, 0:w] = overlayList[totalFingers-1]

        
    # 04. 載入圖片至影像中
    #h, w, c = overlayList[0].shape
    #img[0:h, 0:w] = overlayList[0]

    # 05. 顯示FPS (frame rate)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS:{int(fps)}', (wCam-120, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)

    
    cv2.imshow("Image", img)
    # 01. 按Q離開
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


['1.png', '2.png', '3.png', '4.png', '5.png', '6.png']
Fingerimages/1.png
Fingerimages/2.png
Fingerimages/3.png
Fingerimages/4.png
Fingerimages/5.png
Fingerimages/6.png
6


In [2]:
import cv2
import mediapipe as mp
import time

class handDetector():
    def __init__(self, mode=False, maxHands = 2, model_comp = 1, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHnads = maxHands
        self.model_comp = model_comp
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        # 定義手的資訊變數
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHnads, self.model_comp, self.detectionCon, self.trackCon)
        
        # 定義畫圖單元
        self.mpDraw = mp.solutions.drawing_utils
        
        # 09. 定義手指頭的編號
        self.tipIds = [4, 8, 12, 16, 20] 

    
    def findHands(self, img, draw = False):
        # 取得multi_hand_landmarks的資料 
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        #print(results.multi_hand_landmarks)

        # 劃出關鍵點
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)  # 劃出關鍵點間的連結
        return img
     def findPosition(self, img, handNo=0, draw=True):
        # 06. 新增兩個list紀錄x與y的資訊 bbox紀錄邊界範圍與 bbox紀錄範圍
        xList = []
        yList = []
        bbox = []
        
        self.lmList = []   # 06. 將lmList修改為self.lmList

        lmList = []
        if self.results.multi_hand_landmarks:
            if (handNo == -1) :
                for hid, myHand in enumerate(self.results.multi_hand_landmarks):
                    for id, lm in enumerate(myHand.landmark):
                        #print(id, lm)  #印出檢查關鍵點座標
                        h, w, c = img.shape
                        cx, cy=int(lm.x*w), int(lm.y*h)
                        
                        # 06. 將 cx 與 cy 紀錄至 xList 與 yList
                        xList.append(cx)
                        yList.append(cy)

                        #print(id, cy, cx)  #印出轉換後的關鍵點座標
                        
                        self.lmList.append([id, cx, cy])    # 06. 將lmList修改為self.lmList

                        lmList.append([id, cx, cy])
                        if draw:
                            cv2.circle(img, (cx, cy), 5, (hid*100+100, 255-hid*100, 255), cv2.FILLED)
            else:
                if len(self.results.multi_hand_landmarks) > handNo:
                    myHand = self.results.multi_hand_landmarks[handNo]
                    for id, lm in enumerate(myHand.landmark):
                        #print(id, lm)  #印出檢查關鍵點座標
                        h, w, c = img.shape
                        cx, cy=int(lm.x*w), int(lm.y*h)
                        xList.append(cx)
                        yList.append(cy)
                        #print(id, cy, cx)  #印出轉換後的關鍵點座標
                        self.lmList.append([id, cx, cy])
                        if draw:
                            cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        # 06. 由 xlist 與 ylist 中找到最大與最小值
        if len(xList) != 0:
            xmin, xmax = min(xList), max(xList)
        else:
            xmin, xmax = 0, -20   #設定最小位置，最大值-20，因為顯示時+20
        if len(yList) != 0:
            ymin, ymax = min(yList), max(yList)
        else:
            ymin, ymax = 0, -20   #設定最小位置，最大值-20，因為顯示時+20

        # 06. 設定手的邊界範圍
        bbox = xmin, ymin, xmax, ymax
        
        #print (bbox)
        if draw:
            cv2.rectangle(img, (bbox[0]-20, bbox[1]-20), (bbox[2]+20, bbox[3]+20), (0, 255, 0), 3)

        return self.lmList, bbox
    # 09. 增加檢查手指是否向上的函式
    def fingersUp(self):
        fingers = []
        
        # Thumb
        if self.lmList[1][1] > self.lmList[0][1]:
            if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0]-1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
        else:
            if self.lmList[self.tipIds[0]][1] < self.lmList[self.tipIds[0]-1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
                            
        # 4 Fingers
        for id in range(1,5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id]-2][2]:
                fingers.append(1)     
            else:
                fingers.append(0)    
        
        return fingers


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 20)

In [ ]:
import cv2
import time
import os
import HandTrackingModule as htm    # 07. 導入自訂的HandTrackingModule.py

# 01. 取得鏡頭影像的基本架構
wCam, hCam = 640, 480

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

# 02. 設定手指圖片路徑
folderPath = "Fingerimages"
myList = os.listdir(folderPath)
print(myList)

# 03. 載入圖片至overlayList中
overlayList = []
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    print(f'{folderPath}/{imPath}')
    overlayList.append(image)
print(len(overlayList))

# 05. FPS時間設定
pTime = 0

while True:
    success, img = cap.read()
    
    # 04. 水平翻轉畫面
    img = cv2.flip(img, 1) 
    
    # 04. 載入圖片至影像中
    h, w, c = overlayList[0].shape
    img[0:h, 0:w] = overlayList[0]

    # 05. 顯示FPS (frame rate)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS:{int(fps)}', (wCam-120, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)

    
    cv2.imshow("Image", img)
    # 01. 按Q離開
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


In [5]:
import cv2
import time
import os
import HandTrackingModule as htm    # 07. 導入自訂的HandTrackingModule.py



# 05. FPS時間設定
pTime = 0

# 07. 使用 HandTrackingModule.py函式擷取手部資訊
detector = htm.handDetector(detectionCon=0.75, maxHands=1)

# 01. 取得鏡頭影像的基本架構
wCam, hCam = 640, 480

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

# 02. 設定手指圖片路徑
folderPath = "Fingerimages2"
myList = os.listdir(folderPath)
print(myList)

# 03. 載入圖片至overlayList中
overlayList = []
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    print(f'{folderPath}/{imPath}')
    overlayList.append(image)
print(len(overlayList))

while True:
    success, img = cap.read()
    
    # 04. 水平翻轉畫面
    img = cv2.flip(img, 1) 
    
    # 07. 偵測手並傳回座標位置
    img = detector.findHands(img, draw = False)
    lmList, bbox = detector.findPosition(img)

    # 10. 偵測手並傳回座標位置
    if len(lmList) != 0: 
        # 10. 檢查那些手指向上
        fingers = detector.fingersUp()
        #print(fingers) (if 1~9)
        if fingers[1]==1:
            cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
            cv2.putText(img, f'{int(1)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)
            h, w, c = overlayList[0].shape
            img[0:h, 0:w] = overlayList[0]

        if fingers[1]==1 and fingers[2]==1:
            cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
            cv2.putText(img, f'{int(2)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)
            h, w, c = overlayList[1].shape
            img[0:h, 0:w] = overlayList[1]

        if fingers[1]==1 and fingers[2]==1 and fingers[3]==1 :
            cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
            cv2.putText(img, f'{int(3)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)
            h, w, c = overlayList[2].shape
            img[0:h, 0:w] = overlayList[2]

        if fingers[1]==1 and fingers[2]==1 and fingers[3]==1 and fingers[4]==1:
            cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
            cv2.putText(img, f'{int(4)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)
            h, w, c = overlayList[3].shape
            img[0:h, 0:w] = overlayList[3]
        
        if fingers[0]==1:
            cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
            cv2.putText(img, f'{int(5)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)
            h, w, c = overlayList[4].shape
            img[0:h, 0:w] = overlayList[4]
        
        if fingers[0]==1 and fingers[1]==1:
            cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
            cv2.putText(img, f'{int(6)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)
            h, w, c = overlayList[5].shape
            img[0:h, 0:w] = overlayList[5]
        
        if fingers[0]==1 and fingers[1]==1 and fingers[2]==1:
            cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
            cv2.putText(img, f'{int(7)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)
            h, w, c = overlayList[6].shape
            img[0:h, 0:w] = overlayList[6]
       
        if fingers[0]==1 and fingers[1]==1 and fingers[2]==1 and fingers[3]==1:
            cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
            cv2.putText(img, f'{int(8)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)
            h, w, c = overlayList[7].shape
            img[0:h, 0:w] = overlayList[7]
       
        if fingers[0]==1 and fingers[1]==1 and fingers[2]==1 and fingers[3]==1 and fingers[4]==1:
            cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
            cv2.putText(img, f'{int(9)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)
            h, w, c = overlayList[8].shape
            img[0:h, 0:w] = overlayList[8]
            
        if fingers[0]==0 and fingers[1]==0 and fingers[2]==0 and fingers[3]==0 and fingers[4]==0:
            cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
            cv2.putText(img, f'{int(0)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)
            h, w, c = overlayList[-1].shape
            img[0:h, 0:w] = overlayList[-1]
        # 10. 計算向上手指數量
        totalFingers = fingers.count(1)   
        
        
        # 10. 輸出數字
        #cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
        #cv2.putText(img, f'{int(totalFingers)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)
    
    
        # 11. 修改原4.載入圖片至影像中的程式並放入if len(lmList) != 0:中
       # h, w, c = overlayList[-1].shape
        #img[0:h, 0:w] = overlayList[totalFingers-1]


    # 05. 顯示FPS (frame rate)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS:{int(fps)}', (wCam-120, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)

    cv2.imshow("Image", img)
    # 01. 按Q離開
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


['01.png', '02.png', '03.png', '04.png', '05.png', '06.png', '07.png', '08.png', '09.png', '10.png']
Fingerimages2/01.png
Fingerimages2/02.png
Fingerimages2/03.png
Fingerimages2/04.png
Fingerimages2/05.png
Fingerimages2/06.png
Fingerimages2/07.png
Fingerimages2/08.png
Fingerimages2/09.png
Fingerimages2/10.png
10


In [1]:
import cv2
import time
import os,numpy as np
import HandTrackingModule as htm    # 07. 導入自訂的HandTrackingModule.py

# 01. 取得鏡頭影像的基本架構
wCam, hCam = 640, 480

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

# 02. 設定手指圖片路徑
folderPath = "D:\\python\\cv2\\finger\\Fingerimages2"
myList = os.listdir(folderPath)
print(myList)

# 03. 載入圖片至overlayList中
overlayList = []
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    print(f'{folderPath}/{imPath}')
    overlayList.append(image)
print(len(overlayList))

# 05. FPS時間設定
pTime = 0

# 07. 使用 HandTrackingModule.py函式擷取手部資訊
detector = htm.handDetector(detectionCon=0.75)


while True:
    success, img = cap.read()

    # 04. 水平翻轉畫面
    #img = cv2.flip(img, 1) 

    # 07. 偵測手並傳回座標位置
    img = detector.findHands(img, draw = False)
    lmList, bbox = detector.findPosition(img,handNo=-1 ,draw=True)
    #print(np.shape(lmList))
    #print(lmList)
    # 10. 偵測手並傳回座標位置
    if len(lmList) != 0: 
        # 10. 檢查那些手指向上
        fingers = detector.fingersUp()
        #print(fingers)
        img=detector.HW16(img,fingers,overlayList)
        # 10. 計算向上手指數量
        totalFingers = fingers.count(1)
        # 10. 輸出數字
        #cv2.rectangle(img, (20, 225), (170, 425), (255,0,255), cv2.FILLED)
        #cv2.putText(img, f'{int(totalFingers)}', (45, 375), cv2.FONT_HERSHEY_COMPLEX, 5, (255,0,0), 15)
        # 11. 修改原4.載入圖片至影像中的程式並放入if len(lmList) != 0:中
        #h, w, c = overlayList[totalFingers-1].shape
        #img[0:h, 0:w] = overlayList[totalFingers-1]
    # 04. 載入圖片至影像中
    #h, w, c = overlayList[0].shape
    #img[0:h, 0:w] = overlayList[0]
    # 05. 顯示FPS (frame rate)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS:{int(fps)}', (wCam-120, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)
    cv2.imshow("Image", img)
    # 01. 按Q離開
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'cv2'